In [1]:
%load_ext autoreload
%autoreload 2

In [51]:
import sys_pdf
import pdf_functions as pdf
import os
import sys_pdf

'''
Process a PDF using search dictionary
'''

def open_and_read(working_dir, file_name):# open a file and read
    try: 
        path_name = working_dir + "/" + file_name
        scanned_text = pdf.read_pdf(path_name)
    except:
        sys_pdf.sys_log_entry('read error', "Could not open file: " + path_name)
        sys_pdf.move_file(working_dir, "not_processed", file_name)
    return scanned_text, path_name


def format_text(scanned_text):
    # add the contents for each page into a list
    contents = pdf.text_to_list(scanned_text)

    # iterate over each page
    by_line = []
    for page in range(0, len(scanned_text.pages)):  # create a flat list of strings
        by_line += contents[page].split('\n')       # remove the line breaks
    return by_line


def pull_name_date(overview_table):
    company_name = overview_table['invoice_company'][0].replace('.', '_')
    # format dates to standard YYYY-MM-DD
    formatted_date_str = pdf.format_date(overview_table['invoice_date'][0], '%B %d, %Y') # November 11, 2015
    return company_name, formatted_date_str

def save_name(company_name, formatted_date_str):
    name_base = company_name + "_" + formatted_date_str
    return name_base

def process_pdfs(working_dir, csv_dir):
    file_counter = 0
     
    # for filename in working_dir
    for file_name in os.listdir(working_dir):
    
        # pull text from pdf file
        scanned_text, path_name = open_and_read(working_dir, file_name)
        
        # process the file
        try: 
            # make a list of text
            by_line = format_text(scanned_text)
            
            # open dictionary of search terms
            table_1_search, table_1_type = pdf.table_1_search_dict()
            table_2_search, table_2_type = pdf.table_2_search_dict()
        
            # create tables with pulled data
            table_1 = pdf.one_table(table_1_search, by_line)
            table_2 = pdf.many_table(table_2_search, by_line)  
        
            # 
            table_1_df = pdf.one_table_df(table_1)
            table_2_df = pdf.many_table_df(table_2)  # verifies that the length of each value list is the same in a many table
        
            # add a column of invoice numbers for each table_2 entry
            #table_2_df['invoice_number'] = table_1['invoice_number'][0]
            table_2_df.insert(0, 'invoice_number', table_1['invoice_number'][0])

            # grab company name and invoice date from overview table
            company_name, invoice_date = pull_name_date(table_1)
            name_base = save_name(company_name, invoice_date)
            
            # save the csv and log
            table_dict = {"overview":table_1_df, "products":table_2_df}
            for key, value in table_dict.items():
                routine,log_value=pdf.save_as_csv(value, csv_dir, name_base, key)
                sys_pdf.sys_log_entry(routine, log_value)
            
            # save the pdf in the 'processed' directory
            sys_pdf.move_file(working_dir, "processed", file_name)
            
            file_counter += 1
            
        except:
            sys_pdf.sys_log_entry('production error', "Could not properly analyze file: " + path_name)
            #  move pdf to error directory
            sys_pdf.move_file(working_dir, "not_processed", file_name)
    
    print(f"A total of {file_counter} PDF files were processed.")









In [52]:
# Move PDF files to working directory
source_dir = "pdf_inbox"
working_dir = "working_dir"
csv_dir = "example_output_files"

files_present = sys_pdf.to_working_dir(source_dir, working_dir)

if files_present:   # only move forward with formatting if there are files to process
    process_pdfs(working_dir, csv_dir)

move Files in inbox: 1. Moved to working dir: 1. 
The file has 2 pages.
CSV OK. Saved: overview Amazon_com_2015-11-11
CSV OK. Saved: products Amazon_com_2015-11-11
A total of 1 PDF files were processed.


In [94]:
'''
Build a search dictionary.
'''
import pandas as pd

# Move PDF files to working directory
source_dir = "NEW_PDF_style"
working_dir = "working_dir"
search_file_name = "amazon111924.xlsx"

# checks directory if is a new PDF file to be formatted, if s move to working dir
files_present = sys_pdf.to_working_dir(source_dir, working_dir)

if files_present:
    # open excel file as a pandas dataframe
    guide = pd.read_excel("search_guides/"+search_file_name)
    
    # table headers from the guide
    overview_table_headers = [ 'Invoice date', 'Invoice number', 'Company name', 'Invoice total' ]
    products_table_headers = [ 'Product name', 'Product company', 'Product amount' ]

    # the name of the pdf file from search guide
    pdf_file = guide['File name of PDF'][0]
    
    # open and pull text from pdf file
    scanned_text, path_name = open_and_read(working_dir, pdf_file)

    # prep the text strings for scanning
    by_line = format_text(scanned_text)

    # iterate over each table header
    #for header in overview_table_headers:
    header = guide['Invoice number'][0]
    
    # iterate over each line in the list of text strings
    for line in by_line:
        # is there a match between the search string and the line?
        # if so return the match
        #matching_string = pdf.find_string(line, header, position, del_chars)

    if matching_string != None:
        # clean up the string
        clean_string = clean_text(matching_string)

move Files in inbox: 1. Moved to working dir: 1. 
The file has 2 pages.


In [100]:
guide['Invoice number'][0]

'104-1523806-7547463'

In [136]:
pos=0
header = guide['Invoice number'][0]
len_target_text = len(header)
for line in by_line:
    char_location = line.find(header)
    if char_location >= 0: 
        the_line = line
        pos=char_location
        break


In [137]:
the_line

'Details for Orde r #104-1523806-7547463 '

In [138]:
pos

20

In [132]:
# The search term 
front = the_line[0:pos]

back = the_line[pos+len_search_str:]

In [139]:
the_line[0:pos]

'Details for Orde r #'

In [132]:
import PyPDF2
import pandas as pd
import os
import openai
from dotenv import load_dotenv
import pdf_functions as pdf
import sys_pdf


class OpenaiExtraction:
    '''
    Use openai api to extract data from a PDF file and format into a table.
    '''

    def __init__(self, working_dir, csv_dir):
        self.working_dir = working_dir
        self.csv_dir = csv_dir
        self.company_name = ""
        self.invoice_date = ""
        self.raw_text = ""
        self.table_dict = {}
        self.extracted_dict = {}
        self.file_counter = 0
        
        # init methods
        load_dotenv()   # Load environment variables from the .env file
        self.client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))    # instantiate openai


    def run_extraction(self):
        # for filename in working_dir
        for file_name in os.listdir(self.working_dir):
            
            # Extract text from the PDF
            self.extract_text_from_pdf(file_name)

            # Build the prompts
            self.build_prompts()
            
            for table_name, prompt in self.table_dict.items():
                
                # Extract specific invoice details using OpenAI ChatCompletion
                response, extracted_data = self.extract_invoice_data(prompt)
                
                # log the number of tokens spent
                sys_pdf.openai_token_log(table_name, response.usage.total_tokens)
            
                # Create a pandas dataframe with the extracted data
                invoice_table = self.create_invoice_table(extracted_data)
                self.extracted_dict.update({table_name:[response, extracted_data, invoice_table]})

                # grab company name and invoice date from overview table
                if table_name == 'overview':
                    self.pull_name_date(invoice_table)

                # save data as csv and log
                routine,log_value = pdf.save_as_csv(invoice_table, self.csv_dir, self.name_base, table_name)
                sys_pdf.sys_log_entry(routine, log_value)

            # save the original pdf in the 'processed' directory
            sys_pdf.move_file(self.working_dir, "processed", file_name)
            
            self.file_counter += 1

 
    def extract_text_from_pdf(self, file_path):
        """Extracts text from a PDF file."""
        text = ""
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text += page.extract_text()
        self.raw_text = text

        
    def build_prompts(self):
        # Prompt for the model to identify relevant data for the OVERVIEW table
        self.overview_prompt = f"""
        Extract the following details from the text:
        - Invoice Date
        - Invoice Number
        - Invoice Company
        - Invoice Total
        Text: {self.raw_text}
        Provide the data in a JSON format with keys: invoice_date, invoice_number, invoice_company, invoice_total.
        """

        # Prompt for the model to identify relevant data for the PRODUCTS table
        self.product_prompt = f"""
        Extract the following details from the text:
        - Invoice Number
        - Product Name
        - Product Seller
        - Price of Product
        Text: {self.raw_text}
        There may be multiple products in the text. 
        Provide the data in a JSON format with keys: invoice_number, product_name, product_seller, product_amount.
        """

        # store these prompts in a dictionary
        self.table_dict = { 'overview': self.overview_prompt, 'product': self.product_prompt }

    
    def extract_invoice_data(self, prompt):
        """Extracts specific invoice details using OpenAI ChatCompletion API."""
        
        #response = client.completions.create()   - this is the depreciated method
        response = self.client.chat.completions.create(
            max_tokens = 3000,
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that extracts structured data from text."},
                {"role": "user", "content": prompt}
            ]
        )
        extracted_data = response.choices[0].message.content.strip("```json").replace("\n","")
        #self.extracted_tables.update({table_name, [extracted_data, response]})  # store results in a dictionary
        return response, eval(extracted_data)  # Convert string to dictionary
    
    def create_invoice_table(self, data):
        """Creates a table with the extracted data."""
        if type(data) == list:
            df = pd.DataFrame(data)
        else:
            df = pd.DataFrame([data])
        #df.columns = ["Invoice Date", "Invoice Number", "Invoice Company", "Invoice Total"]
        return df

    def pull_name_date(self, overview_table):
        # record company name
        self.company_name = overview_table['invoice_company'][0].replace('.', '_')
        # format dates to standard YYYY-MM-DD
        formatted_date_str = pdf.format_date(overview_table['invoice_date'][0], '%B %d, %Y') # November 11, 2015
        self.name_base = self.company_name + "_" + formatted_date_str

In [37]:
import pandas as pd

In [6]:
import sys_pdf
import pdf_functions as pdf

source_dir = "pdf_inbox"
working_dir = "working_dir"
csv_dir = "example_output_files"

# check if files present in the 
files_present = sys_pdf.to_working_dir(source_dir, working_dir)

if files_present:

    # instantiate openai_extraction with directory locations
    charlie = pdf.OpenaiExtraction(working_dir, csv_dir)

    # run the extraction method
    charlie.run_extraction()
    

move Files in inbox: 1. Moved to working dir: 1. 
openai:  overview 815
CSV OK. Saved: overview Amazon_com_2015-11-11
openai:  product 1064
CSV OK. Saved: product Amazon_com_2015-11-11


In [7]:
charlie.extracted_dict['product'][2]

,invoice_number,product_name,product_seller,product_amount
0,104-1523806-7547463,"Honeywell True HEPA Allergen Remover, 465 Sq F...",Amazon.com LLC,$224.00
1,104-1523806-7547463,"Honeywell True HEPA Allergen Remover, 310 Sq F...",Amazon.com LLC,$162.35
2,104-1523806-7547463,Brainwavz Delta Black IEM Earphones With Remot...,Brainwavz Audio,$22.50
3,104-1523806-7547463,Panasonic RPTCM125W Headphones (White),Amazon.com LLC,$9.95


In [6]:
charlie

In [39]:
pd.DataFrame(charlie.extracted_dict['product'][1])

,invoice_number,products
0,104-1523806-7547463,{'product_name': 'Honeywell True HEPA Allergen...
1,104-1523806-7547463,{'product_name': 'Honeywell True HEPA Allergen...
2,104-1523806-7547463,{'product_name': 'Brainwavz Delta Black IEM Ea...
3,104-1523806-7547463,{'product_name': 'Panasonic RPTCM125W Headphon...


In [ ]:
z = {}
z.update({'this':1234})
z.update({'that':4444})
z

In [126]:
if type(ai_extract.extracted_dict['overview'][1]) == list:
    

dict

In [128]:
pd.DataFrame([ai_extract.extracted_dict['overview'][1]])

,invoice_date,invoice_number,invoice_company,invoice_total
0,"November 11, 2015",104-1523806-7547463,Amazon.com,$379.29


In [3]:
Certainly! Here's the structured data extracted from the text in JSON format:```json{  "invoice_number": "104-1523806-7547463",  "products": [    {      "product_name": "Honeywell True HEPA Allergen Remover, 465 Sq Ft, HPA300",      "product_seller": "Amazon.com LLC",      "product_amount": "$224.00"    },    {      "product_name": "Honeywell True HEPA Allergen Remover, 310 Sq Ft, HPA204",      "product_seller": "Amazon.com LLC",      "product_amount": "$162.35"    },    {      "product_name": "Brainwavz Delta Black IEM Earphones With Remote & Mic For Apple iPhone, iPad, iPod & Other Apple iOS Devices",      "product_seller": "Brainwavz Audio",      "product_amount": "$22.50"    },    {      "product_name": "Panasonic RPTCM125W Headphones (White)",      "product_seller": "Amazon.com LLC",      "product_amount": "$9.95"    }  ]}```This JSON structure includes the invoice number and an array of products with their respective names, sellers, and amounts.

SyntaxError: unterminated string literal (detected at line 1) (2542352264.py, line 1)

In [17]:
"there waesd '''thsi that''' other"

"there waesd '''thsi that''' other"

In [31]:
str(charlie.extracted_dict['overview'][0].choices[0].message.content).strip("```")

'json\n{\n  "invoice_date": "November 11, 2015",\n  "invoice_number": "104-1523806-7547463",\n  "invoice_company": "Amazon.com",\n  "invoice_total": "$379.29"\n}\n'